In [29]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [30]:
data = pd.read_csv('raw.csv', sep=',',engine='python')
df = data
len(df)

5776

In [39]:
data_new = df.dropna(subset=['ASH','INSUL','TAAV','RH','dayav_ta','dayav_rh','MET'])
data_new.to_csv('raw_new.csv')
# get features and thermal sensation
y = data_new['ASH']
x = data_new[['INSUL','TAAV','RH','dayav_ta','dayav_rh','MET']]
#bool = (y==-3 or y==-2 or y==-1 or y==0 or y==1 or y==2 or y==3)
y = np.round(y)
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y)
np.unique(encoded)

array([0, 1, 2, 3, 4, 5, 6])

In [32]:
import sklearn.preprocessing, sklearn.decomposition,sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import datasets, svm
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn_pandas import DataFrameMapper
from sklearn.ensemble import RandomForestClassifier

In [33]:
mapper = DataFrameMapper([(['INSUL'], None),
                         (['TAAV'], None),
                          (['RH'], None),
                          (['dayav_ta'], None),
                         (['dayav_rh'],None),
                         (['MET'],None)])
mapper.fit_transform(x.copy())
# count the number of thermal sensation
# bool = (y==3)
# len(y[bool])

array([[  0.7842    ,  25.        ,  61.        ,  27.55555556,
         81.55      ,   1.        ],
       [  0.682944  ,  25.        ,  61.        ,  27.55555556,
         81.55      ,   1.        ],
       [  0.682944  ,  25.        ,  61.        ,  27.55555556,
         81.55      ,   1.        ],
       ..., 
       [  0.82828   ,  23.3       ,  56.1       ,  26.94444444,
         49.5       ,   1.        ],
       [  0.79272   ,  23.2       ,  56.1       ,  26.94444444,
         49.5       ,   1.        ],
       [  0.65324   ,  23.4       ,  53.5       ,  26.94444444,
         49.5       ,   1.        ]])

In [34]:
#clf = svm.SVC(kernel='linear')
C=1
clf = svm.SVC(kernel='rbf',gamma=0.1,C=C)
pipe = sklearn.pipeline.Pipeline([('featurize', mapper),('svc', clf)])
# random forest
clf = RandomForestClassifier(n_estimators=50, max_depth = 3, random_state=0)
pipe = sklearn.pipeline.Pipeline([('featurize', mapper),('randomforest', clf)])
#np.round(cross_val_score(pipe, X=data_new.copy(), y=data_new.comfort, scoring='r2'), 2)
cross_val_score(pipe, X=x.copy(), y=encoded, cv=5)

array([ 0.38145695,  0.41644562,  0.4873502 ,  0.488     ,  0.48796791])

In [35]:
# testing
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

In [36]:
# evaluation
predicted = cross_val_predict(clf, x.copy(), encoded, cv=5)
metrics.accuracy_score(encoded, predicted) 

0.45210218201170838

In [37]:
test = x.iloc[[0]]
clf.fit(x,y)
gt = y[0]
pred = clf.predict(test)
pred
gt

0.0

In [38]:
import pickle
filename = 'svm_model.sav'
pickle.dump(clf,open(filename,'wb'),protocol=2)
load_model = pickle.load(open(filename,'rb'))
pred = load_model.predict(test)
pred

array([ 0.])